# GenAI-Camp: Day 03
## Lesson: Next RAG

This lesson is intended to show you how to further extend a RAG system.

During this lesson you will learn how to ...

- use a fact-checking guardrail
- use rag as a tool

### Set up the environment
Import the necessary libraries, set constants, and define helper functions.

In [ ]:
from pydantic import BaseModel
import json
from google import genai
from google.genai import types
import os
import requests
import json
from enum import Enum

In [ ]:
if os.getenv("COLAB_RELEASE_TAG"):
   from google.colab import userdata
   GOOGLE_API_KEY=userdata.get('GEMINI_API_KEY')
   COLAB = True
   print("Running on COLAB environment.")
else:
   from dotenv import load_dotenv, find_dotenv
   load_dotenv(find_dotenv())
   GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")
   JINA_API_KEY = os.getenv("JINA_API_KEY")
   COLAB = False
   print("WARNING: Running on LOCAL environment.")
client = genai.Client(api_key=GOOGLE_API_KEY)

In [ ]:
# Install additional libraries
if COLAB:
  !pip install -qU chromadb
    
# Import additional libraries
from chromadb import PersistentClient

In [ ]:
# Define path of ressources
if COLAB:
    # Clone the data repository into colab
    !git clone https://github.com/openknowledge/workshop-genai-camp-data.git
    %cd workshop-genai-camp-data
    !git lfs pull 
    ROOT_PATH = "/content/workshop-genai-camp-data/day-03"
else:
    ROOT_PATH = ".."
DATA_PATH = ROOT_PATH + "/data"
EVALUATION_PATH = ROOT_PATH + "/evaluation"
KNOWLEDGEBASE_PATH = ROOT_PATH + "/knowledgebase"
BOOK_CATALOG_FILE = DATA_PATH + "/books.json"
TESTDATA_FILE = EVALUATION_PATH + "/synthetic_testset.json"

In [ ]:
# Set default models
GENERATION_MODEL = "gemini-1.5-flash"
EMBEDDING_MODEL = "models/text-embedding-004"

# Set default values for model, model parameters and prompt
DEFAULT_CONFIG_TEMPERATURE = 0.9 
DEFAULT_CONFIG_TOP_K = 1
DEFAULT_CONFIG_MAX_OUTPUT_TOKENS = 200 
DEFAULT_SYSTEM_PROMPT = "Your are a friendly assistant"
DEFAULT_USER_PROMPT = " "

# Set defaults for rag
DEFAULT_K = 3
DEFAULT_CHUNK_SIZE = 2000
DEFAULT_CHUNK_OVERLAP = 100

In [ ]:
def generate_gemini_completion(
        model_name: str = GENERATION_MODEL, 
        temperature:float = DEFAULT_CONFIG_TEMPERATURE,
        top_k: int = DEFAULT_CONFIG_TOP_K, 
        max_output_tokens: int = DEFAULT_CONFIG_MAX_OUTPUT_TOKENS, 
        system_prompt : str = DEFAULT_SYSTEM_PROMPT, 
        user_prompt : str = DEFAULT_USER_PROMPT,
        verbose: bool = False
        ) -> str: 
    
    """ Calls a gemini model with a given set of parameters and returns the completions 
    
    Parameters
    ----------
    model_name : str, optional [default: DEFAULT_GEMINI_MODEL]
        The name of the model to use for the completion
    temperature : float, optional [default: DEFAULT_CONFIG_TEMPERATURE]
        The temperature of the model
    top_k : int, optional [default: DEFAULT_CONFIG_TOP_K]
        The number of most recent matches to return
    max_output_tokens : int, optional [default: DEFAULT_CONFIG_MAX_OUTPUT_TOKENS]
        The maximum number of output tokens to return
    system_prompt : str, optional [default: DEFAULT_SYSTEM_PROMPT]
        The system prompt to use for the completion
    user_prompt : str, optional [default: DEFAULT_USER_PROMPT]
        The user prompt to use for the completion
    verbose : bool, optional [default: False]
        Whether to print details of the completion process or not. Defaults to False            
    Returns 
    -------
    str :
        the generated text      
    """    
    if verbose: 
        # print out summary of input values / parameters
        print(f'Generating answer for following config:')
        print(f'  - SYSTEM PROMPT used:\n {system_prompt}')
        print(f'  - USER PROMPT used:\n {user_prompt}')
        print(f'  - MODEL used:\n {model_name} (temperature = {temperature}, top_k = {top_k}, max_output_tokens = {max_output_tokens})')

    # create generation config 
    model_config = types.GenerateContentConfig(
        max_output_tokens=max_output_tokens,
        temperature=temperature,
        top_k=top_k,
        system_instruction=system_prompt,
    )
    
    # create generation request
    response = client.models.generate_content(
        model=model_name,
        contents=user_prompt,
        config=model_config,
    )
    
    return response.text

In [ ]:
class Metadata(BaseModel):
    """Represents the metadata of a document which is stored in the knowledgebase."""
    url: str
    title: str
    pub_year: int

class Book(BaseModel):
    """Represents a book with its metadata."""
    metadata: Metadata
    summary: str

In [ ]:
# RAG building blocks

# This will be the chromadb collection we use as a knowledge base. We do not need the client.
chromadb_collection = PersistentClient(path=KNOWLEDGEBASE_PATH).get_or_create_collection(name="default")

class FetchedChunk(BaseModel):
    """Represents a chunk fetched from the knowledgebase."""
    chunk: str
    metadata: Metadata

# Building Block "Embedding": Create multi dimensional embeddings for a given chunk.
def do_embed(chunk: str) -> list[float]:
  """ Embeds a given chunk and returns the embedding

  Parameters
  ----------
  chunk : str
      The chunk to be embedded
  Returns
  -------
  embedding: [float]
      The created embedding
  """
  content_embeddings = client.models.embed_content(model=EMBEDDING_MODEL, contents=chunk).embeddings
  return content_embeddings[0].values

# Building Block "Augmentation": Create an updated prompt by merging the original user input with the provided context
# Attention: We manipulated the augmented prompt in order to see the guardrails in action
def augment(user_input: str, context: list[str]) -> str:
  """ Augments a given user input by merging it with the provided context and returns the augmented prompt

  Parameters
  ----------
  user_input : str
      The user input to be augmented
  context : [str]
      The context to be merged with the user input
  Returns
  -------
  augmented_prompt: str
      The created augmented prompt
  """
  prepared_context = "\n".join(context)
  augmented_prompt = f"""
    Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in
    provided context just say, "answer is not available in the context", don't provide the wrong answer\n\n
    Context:\n{prepared_context}?\n
    Question: \n{user_input}\n

    Answer:
  """
  return augmented_prompt

# Building Block "Top-k Fetching": Get the k semantically closest chunks to the user input from the knowledgebase
def do_top_k_fetching(user_input_embedding: list[float], top_k: int) -> list[FetchedChunk]:
  """ Fetches the k semantically closest chunks to the user input from the knowledgebase

  Parameters
  ----------
  user_input_embedding : [float]
      The embedding of the user input
  top_k : int
      The number of semantically closest chunks to be fetched

  Returns
  -------
  context: [str]
      The fetched chunks
  """
  # Since we will do the fetching always only for one user_input,
  # instead of querying for multiple embeddings simultanously as allowed by the choma API,
  # we add the embeddings below to a list and return only the first document (chunk)
  
  query_result = chromadb_collection.query(
      query_embeddings=[user_input_embedding],
      n_results=top_k,
  )
  chunks = query_result["documents"][0]
  metadatas = query_result["metadatas"][0]
  
  fetched_chunks = []
  for i in range(len(chunks)):
    chunk = chunks[i]
    metadata = metadatas[i]
    fetched_chunk = FetchedChunk(chunk=chunk, metadata=Metadata(**metadata))
    fetched_chunks.append(fetched_chunk)
  return fetched_chunks

# Building Block "Generation": Use the generation model to create a response
def generate_response(prompt: str) -> str:
  """ Generates a response for a given prompt

  Parameters
  ----------
  prompt : str
      The prompt to be used for the generation
  Returns
  -------
  response: str
      The generated response
  """
  return generate_gemini_completion(
      model_name=GENERATION_MODEL,
      user_prompt=prompt,
  )

# The rag function should now return the response and the context in order to be evaluated further
def do_rag(user_input: str, top_k: int) -> tuple[str, list[str]]:
  """ Runs the RAG pipeline with a given user input and returns the response and the context

  Parameters
  ----------
  user_input : str
      The user input to be used for the RAG pipeline
  Returns
  -------
  response: str
      The generated response
  context: [str]
      The fetched chunks
  """
  # Embed the user input
  user_input_embedding = do_embed(chunk=user_input)

  # "R" like "Retrieval": Get the k semantically closest chunks to the user input from the knowledgebase
  fetched_chunks = do_top_k_fetching(user_input_embedding=user_input_embedding, top_k=top_k)
  context = [chunk.chunk for chunk in fetched_chunks]

  # "A" like "Augmented": Create the augmented prompt
  augmented_prompt = augment(user_input=user_input, context=context)

  # "G" like "Generation": Generate a response
  response = generate_response(prompt=augmented_prompt)

  return (response, fetched_chunks)

In [ ]:
# Define classes and convenient functions for function calls
class FunctionCall(BaseModel):
    """
    Function call model for Gemini API.
    """
    name: str
    arguments: dict


class GeminiResponse(BaseModel):
    """
    Gemini response model.
    """
    text: str | None
    function_call: FunctionCall | None


class MimeType(Enum):
    """
    Enum for MIME types.
    """
    JSON = "application/json"


class ResponseFormat(BaseModel):
    """
    Response format model for Gemini API.
    """
    response_mime_type: MimeType
    response_schema: type
    
   
def enhanced_generate_gemini_completion(
        user_prompt : str,
        response_format: ResponseFormat | None = None,
        system_prompt: str = DEFAULT_SYSTEM_PROMPT,
        model_name: str = GENERATION_MODEL, 
        function_declarations: list | None = None,
        verbose: bool = False
        ): 
    """
    Call the GenAI model with function declarations and return the response.
    Args:
        user_prompt (str): The prompt to send to the model.
        response_format (ResponseFormat): The format of the response.
        system_prompt (str): The system prompt to use.
        model_name (str): The name of the model to use.
        function_declarations (list): List of function declarations.
        verbose (bool): If True, print the response.
    Returns:
        GeminiResponse: The response from the model.
    """

    # Configure tools
    tools = []
    if function_declarations:
        tools.append(types.Tool(function_declarations=function_declarations))

    # Configure response format
    response_schema = None
    response_mime_type = None
    if response_format:
        response_schema = response_format.response_schema
        response_mime_type = response_format.response_mime_type.value

    config = types.GenerateContentConfig(
        tools=tools,
        system_instruction=system_prompt,
        response_schema=response_schema,
        response_mime_type=response_mime_type,
    )

    # Send request with function declarations
    response = client.models.generate_content(
        model=model_name,
        contents=user_prompt,
        config=config,
    )

    function_call = None
    if response.candidates[0].content.parts[0].function_call:
        function_call_gemini = response.candidates[0].content.parts[0].function_call
        function_call = FunctionCall(
            name=function_call_gemini.name,
            arguments=function_call_gemini.args,
        )

    if verbose:
        print(f"Response: {response}")
    
    return GeminiResponse(
        text=response.candidates[0].content.parts[0].text,
        function_call=function_call,
    )

### Exercise 01: Fact-Check
Fact Checking is an important feature of RAG systems. Your task is to update the `do_rag` function in order to  check if the answer is grounded in the context. You should use a LLM-as-a-judge approach. For this use a separate call to the gemini model and create a prompt, which should check if the original answer is grounded in the retrieved context.  
**Hints**:
* You can use the enhaced gemini completion method, if you need response formatting

In [ ]:
# TODO: Your implementation

### Exercise 02: RAG-as-a-tool
RAG systems are useful for a lot of usecases, but sometimes the user just wants some general advice, which ist not based on internal knowldge. As a last exercise, you should provide the rag system as a tool to the model. Create a separate function, which encapsulates the RAG logic. Write a function declaration for this, and call the tool if neccessary. 

In [ ]:
# As a reminder, this is how you declare a function for tool calling:
weather_function = {
    "name": "get_weather",
    "description": "Get the current weather for a given location.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "The location for which to get the weather.",
            }
        },
        "required": ["location"],
    },
}

In [ ]:
# TODO: Your implementation